# Explicación del Método para Calcular el Scorecard

## ¿Qué es un Scorecard?
Un **scorecard** es una herramienta utilizada en modelos de riesgo crediticio para transformar las características de un cliente en un puntaje interpretable. Este puntaje indica la probabilidad de incumplimiento y se calcula utilizando el análisis estadístico de las características del cliente en relación con el objetivo (por ejemplo, si incumple o no).

---

## Pasos del Método

### 1. Calcular el Weight of Evidence (WoE)
El **Weight of Evidence (WoE)** mide la diferencia relativa entre la proporción de incumplidores (**Bad Rate**) y no incumplidores (**Good Rate**) dentro de cada categoría de una característica.

#### Fórmula del WoE:
$$
\text{WoE} = \ln\left(\frac{\text{Proporción de No Incumplidores \, (\text{Good Rate})}}{\text{Proporción de Incumplidores \, (\text{Bad Rate})}}\right)
$$

#### Pasos para calcular el WoE:
1. **Dividir los datos**:
   - Para cada categoría de la característica, calcular:
     - El número de no incumplidores:
       $$
       \text{Good} = \text{Número total de no incumplidores en la categoría}
       $$
     - El número de incumplidores:
       $$
       \text{Bad} = \text{Número total de incumplidores en la categoría}
       $$

2. **Calcular las proporciones**:
   - Proporción de no incumplidores:
     $$
     \text{Good Rate} = \frac{\text{Good}}{\text{Total Good}}
     $$
   - Proporción de incumplidores:
     $$
     \text{Bad Rate} = \frac{\text{Bad}}{\text{Total Bad}}
     $$

3. **Evitar divisiones por cero**:
   - Si $ \text{Bad Rate} = 0 $ o $ \text{Good Rate} = 0 $, se utiliza un pequeño valor de corrección $ \epsilon $.

4. **Aplicar la fórmula**:
   - Sustituir los valores en la fórmula para calcular el WoE:
     $$
     \text{WoE} = \ln\left(\frac{\text{Good Rate} + \epsilon}{\text{Bad Rate} + \epsilon}\right)
     $$

---

### 2. Calcular el Information Value (IV)
El **Information Value (IV)** mide la capacidad predictiva de una característica. Es la suma del producto entre la diferencia de proporciones y el WoE para cada categoría.

#### Fórmula del IV:
$$
\text{IV} = \sum_{i=1}^k (\text{Good Rate}_i - \text{Bad Rate}_i) \cdot \text{WoE}_i
$$
Donde \( k \) es el número de categorías de la característica.

---

### 3. Asignar Puntos al Scorecard
El WoE de cada categoría se transforma en un puntaje utilizando una fórmula basada en un puntaje base (\( \text{base\_score} \)) y un parámetro llamado **Points to Double Odds (PDO)**.

#### Fórmula de los Puntos:
$$
\text{Puntos} = \text{Offset} + \text{Factor} \cdot \text{WoE}
$$
Donde:
- \( \text{Factor} = \frac{\text{PDO}}{\ln(2)} \)
- \( \text{Offset} = \text{base\_score} - \left(\text{Factor} \cdot \ln(\text{Odds Base})\right)
- \text{Odds Base} = \frac{\text{Proporción de Incumplidores}}{\text{Proporción de No Incumplidores}}

---

### 4. Calcular el Puntaje Total
El puntaje total de un cliente es la suma del puntaje asignado a cada una de sus categorías más el puntaje base:
$$
\text{Puntaje Total} = \text{base\_score} + \sum_{i=1}^k (\text{Puntos de la Categoría}_i)
$$




In [2]:
import pandas as pd
import numpy as np

# Cargar el dataset
data = pd.read_csv("cleaned_dataset.csv")
data['loan_status'] = data['loan_status'].apply(lambda x: 1 if x == "Incumple" else 0)

# Características categóricas
categorical_features = ['term', 'sub_grade', 'home_ownership', 'verification_status', 'purpose']

# Calcular WoE e IV
def calculate_woe_iv(data, feature, target):
    lst = []
    epsilon = 1e-6
    for cat in data[feature].unique():
        count = data[data[feature] == cat].shape[0]
        good = data[(data[feature] == cat) & (data[target] == 0)].shape[0]
        bad = data[(data[feature] == cat) & (data[target] == 1)].shape[0]
        good_perc = good / data[data[target] == 0].shape[0] if data[data[target] == 0].shape[0] > 0 else 0
        bad_perc = bad / data[data[target] == 1].shape[0] if data[data[target] == 1].shape[0] > 0 else 0
        good_perc = max(good_perc, epsilon)
        bad_perc = max(bad_perc, epsilon)
        woe = np.log(good_perc / bad_perc)
        iv = (good_perc - bad_perc) * woe
        lst.append({'Category': cat, 'Count': count, 'WoE': woe, 'IV': iv})
    woe_df = pd.DataFrame(lst)
    iv_sum = woe_df['IV'].sum()
    return woe_df, iv_sum

# Crear scorecard
base_score = 600
pdo = 50

def calculate_points(woe, base_score, pdo):
    factor = pdo / np.log(2)
    offset = base_score
    max_woe = 10
    woe = np.clip(woe, -max_woe, max_woe)
    return int(offset + (factor * woe))

# Generar scorecard
woe_iv_dict = {}
scorecard = {}

for feature in categorical_features:
    woe_df, iv = calculate_woe_iv(data, feature, 'loan_status')
    woe_df['Points'] = woe_df['WoE'].apply(lambda x: calculate_points(x, base_score, pdo))
    woe_iv_dict[feature] = {'WoE': woe_df, 'IV': iv}
    scorecard[feature] = woe_df[['Category', 'Points']]

# Calcular puntaje total
def calculate_total_score(row, scorecard):
    total_score = base_score
    for feature, score_df in scorecard.items():
        category = row[feature]
        if category in score_df['Category'].values:
            points = score_df[score_df['Category'] == category]['Points'].values[0]
        else:
            points = 0
        total_score += points
    return total_score

data['Score'] = data.apply(lambda row: calculate_total_score(row, scorecard), axis=1)


KeyboardInterrupt: 

## scorecards guardo en el dataset

In [ ]:


# Guardar el nuevo dataset con la columna de scorecard
new_dataset_path = "dataset_with_scorecard.csv"
data.to_csv(new_dataset_path, index=False)


